In [1]:
!rm  -f ~/.letta/sqlite.db

In [2]:
import letta
from letta.schemas.memory import ChatMemory

import chromadb

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

import os
import sys
super_directory = os.path.abspath('..')
sys.path.append(super_directory)

from prompts import sanjay_persona, system_prompt

In [3]:
# Creating the letta_client and setting default LLM and Embedding Model
letta_client = letta.create_client()

letta_client.set_default_llm_config(letta.LLMConfig.default_config("gpt-4o-mini")) 
letta_client.set_default_embedding_config(letta.EmbeddingConfig.default_config("text-embedding-ada-002")) 

In [4]:
# Setting agent name
agent_name = 'dr_sanjay_sarma'

In [5]:
# Getting the vector_db with Sanjay Sarma's information
chroma_client = chromadb.PersistentClient(path='../chromadb')
sanjay_collection = chroma_client.get_collection(name='sanjay_sarma')

vdb_embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')

vectordb = Chroma(
    collection_name='sanjay_sarma',
    embedding_function=vdb_embed_model,
    persist_directory="../chromadb")

/tmp/ipykernel_22845/2753136756.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [6]:
# Function as tool
def get_sanjay_information(query: str):
    """
    Retrieves relevant information related to a query.

    Args:
        query (str): The query string for which information is being retrieved.

    Returns:
        str: The concatenated content of the relevant search results based on the query.
    """
    from langchain_openai import OpenAIEmbeddings
    from langchain.vectorstores import Chroma
    
    vdb_embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')
    vectordb = Chroma(
        collection_name='sanjay_sarma',
        embedding_function=vdb_embed_model,
        persist_directory="../chromadb")
    
    text = ""
    results = vectordb.similarity_search(query, k=5)
    for result in results:
        text += result.page_content
    return text

get_info_tool = letta_client.create_tool(get_sanjay_information)

In [7]:
# Create sanjay agent
agent_state = letta_client.create_agent(
    name=agent_name,
    tool_ids=[get_info_tool.id],
    memory=ChatMemory(
        human="",
        persona=sanjay_persona
    )
)

print(f"Created agent with name {agent_state.name} and unique ID {agent_state.id}")

Created agent with name dr_sanjay_sarma and unique ID agent-308205a2-d85c-4c86-bba1-af23331551e6


In [8]:
# Updating the system prompt
agent_state.system = system_prompt

In [9]:
# Getting and printing response
user_message = "Tell me all about your degrees"

response = letta_client.send_message(
    agent_id=agent_state.id, 
    message=user_message, 
    role="user" 
)
response

LettaResponse(messages=[ReasoningMessage(id='message-b6a4050a-2087-4ad4-9098-e6adac2a75cf', date=datetime.datetime(2025, 1, 12, 7, 46, 18, 325633, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='The user wants to know about my educational background. I need to provide a brief overview of my degrees and their significance.'), ToolCallMessage(id='message-b6a4050a-2087-4ad4-9098-e6adac2a75cf', date=datetime.datetime(2025, 1, 12, 7, 46, 18, 325633, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='get_sanjay_information', arguments='{\n  "query": "Sanjay Sarma degrees",\n  "request_heartbeat": true\n}', tool_call_id='call_oy8nGPAHtHcBdtDBoPz3ZVL3')), ToolReturnMessage(id='message-990a1b71-dc56-4213-a12d-3175a518ae57', date=datetime.datetime(2025, 1, 12, 7, 46, 18, 781889, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "Sanjay Sarma earned his Bachelor’s degree in mechanical engineering from the Indian  Institute of Technology, Kanpur in 1989, before pursuing advanced studies at Carnegie  Mellon University where he completed his Master\'s degree in 1992, specializing in  operations research. He earned his PhD in mechanical engineering from the University  of California, Berkeley in 1995, focusing on automation and systems optimization.[5] Between degrees, he worked at Schlumberger Oilfield Services in Aberdeen,Sanjay Sarma\'s career path is a remarkable journey that spans academia, innovation,  and entrepreneurship. His extensive experience and contributions to education and  technology have established him as a leading figure in these fields. Below is a detailed  overview of his career trajectory: \\n\\nEarly Career:\\n\\n1. Initial Professional ExperienceSanjay Sarma has received numerous awards and recognitions throughout his  distinguished career, reflecting his significant contributions to academia, technology,  and education. Below is a comprehensive overview of his awards and recognitions: \\n\\nMajor Awards and Honors \\n\\n1. MIT MacVicar Fellowship \\n\\n● Description: This prestigious fellowship is awarded to faculty members at MIT  who demonstrate excellence in undergraduate education. It recognizes  outstanding teaching and mentorship.Since 2023, Sarma has been leading the Asia School of Business, where he integrates  technology, sustainability, and innovation into business education to prepare leaders for  the challenges of a rapidly changing world. \\n\\nEducational Background \\n\\n1. Bachelor\'s Degree \\n\\n● Institution: Indian Institute of Technology (IIT), Kanpur \\n\\n● Field of Study: Mechanical Engineering \\n\\n● Year of Graduation: 1989Recognition \\n\\nSanjay Sarma\'s career has been marked by numerous accolades: \\n\\n● He received the MIT MacVicar Fellowship for excellence in teaching. ● He was honored with the National Science Foundation CAREER Award,  recognizing his contributions to research and education. \\n\\n● Featured by publications like Business Week and Fast Company for his  innovations in technology and education.",\n  "time": "2025-01-12 07:46:18 AM UTC+0000"\n}', status='success', tool_call_id='call_oy8nGPAHtHcBdtDBoPz3ZVL3', stdout=None, stderr=None), ReasoningMessage(id='message-1c81e46c-41cd-4c48-a07d-3a307f1e06b7', date=datetime.datetime(2025, 1, 12, 7, 46, 21, 242778, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='Received information about my degrees. Time to summarize and present it clearly to the user.'), ToolCallMessage(id='message-1c81e46c-41cd-4c48-a07d-3a307f1e06b7', date=datetime.datetime(2025, 1, 12, 7, 46, 21, 242778, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "I earned my Bachelor\'s degree in Mechanical Engineering from the Indian Institute of Technology, Kanpur in 1989. Then, I completed my Master\'s in Operations Research